In [13]:
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

Matplotlib is building the font cache; this may take a moment.


In [14]:
embeddings_dict = {}
with open("data/glove/glove.6B.100d.txt", 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        token = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[token] = vector

In [15]:
def find_closest_embeddings(embedding, cutoff=25):
    return sorted(embeddings_dict.keys(), key=lambda token: spatial.distance.euclidean(embeddings_dict[token], embedding))

In [16]:
print(find_closest_embeddings(
    embeddings_dict["twig"] - embeddings_dict["branch"] + embeddings_dict["hand"]
)[:5])

['flashlight', 'twig', 'clipboard', 'shove', 'hand']


In [46]:
sentence_a = "Here we perform 5-fold cross validation of a KNN model after using a standard scaler"
sentence_b = "In this kernel I present a very simple K-nearest neighbors model based on the quantiles of the distribution"
sentence_c = "And so it begins but you can't have them all. Your heart has to settle down somewhere"

In [47]:
embedding_a = np.zeros_like(embeddings_dict["branch"])
embedding_b = np.zeros_like(embeddings_dict["branch"])
embedding_c = np.zeros_like(embeddings_dict["branch"])

In [48]:
for w in sentence_a.split():
    if w not in embeddings_dict:
        continue
    embedding_a += embeddings_dict[w]

for w in sentence_b.split():
    if w not in embeddings_dict:
        continue
    embedding_b += embeddings_dict[w]

for w in sentence_c.split():
    if w not in embeddings_dict:
        continue
    embedding_c += embeddings_dict[w]

In [50]:
print(find_closest_embeddings(
    embedding_a
)[:5])

['a', 'an', 'this', 'be', 'for']


In [54]:
spatial.distance.euclidean(embedding_a, embedding_b)

23.454050064086914

In [52]:
spatial.distance.euclidean(embedding_a, embedding_c)

32.27524185180664

In [53]:
spatial.distance.euclidean(embedding_b, embedding_c)

30.673566818237305

In [ ]:
spatial.distance.euclidean(embeddings_dict[token], embedding))

In [ ]:
return sorted(embeddings_dict.keys(), key=lambda token: spatial.distance.euclidean(embeddings_dict[token], embedding))